<a href="https://colab.research.google.com/github/tsunufu/emotionJudgment/blob/main/emotionJudgement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>